In [26]:
! pip install -U spacy -q 

In [27]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.2                         
Location         c:\Users\kingu\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy
Platform         Windows-11-10.0.26100-SP0     
Python version   3.12.7                        
Pipelines        pt_core_news_lg (3.8.0)       



In [28]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("pt")
db = DocBin()

In [3]:
import json

TRAIN_DATAS = []

with open('training_data1.json', encoding='utf-8') as f:
    TRAIN_DATAS.append(json.load(f))

with open('training_data2.json', encoding='utf-8') as f:
    TRAIN_DATAS.append(json.load(f))

In [4]:
for TRAIN_DATA in TRAIN_DATAS:
    for text, annot in tqdm(TRAIN_DATA['annotations']): 
        doc = nlp.make_doc(text) 
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents 
        db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 497/497 [00:00<00:00, 1407.29it/s]


Skipping entity


100%|██████████| 501/501 [00:00<00:00, 1829.13it/s]


In [5]:
! python -m spacy init config config.cfg --lang pt --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     36.18    1.51    0.83    8.67    0.02
  0     200      10470.71   4459.56   34.19   49.80   26.04    0.34
  0     400       4362.96   1607.31   36.33   61.40   25.80    0.36
  0     600        392.82   1332.00   47.75   58.99   40.11    0.48
  0     800       4338.11   1486.55   43.81   67.48   32.43    0.44
  1    1000        353.11   1299.13   52.72   63.50   45.06    0.53
  1    1200        133.88   1375.74   54.08   74.25   42.52    0.54
  1    1400        167.28   1524.68   46.66   79.79   32.97    0.47
  2    1600       1898.03   1968.38 

In [22]:
nlp_ner = spacy.load("model-last") 

In [23]:
doc = nlp_ner('''meu celular da Positivo parou de ligar a tela''')

In [24]:
spacy.displacy.render(doc, style="ent", jupyter=True)